Import Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
!pip install numpy
import numpy as np
!pip install z3-solver
from z3 import *
!pip install onnx
import onnx
!pip install onnx2keras
from onnx2keras import onnx_to_keras
from onnx import numpy_helper
!pip install onnxruntime
import onnxruntime
import random
!pip install pandas
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load the ONNX File

In [ ]:
onnx_model = onnx.load("/content/drive/MyDrive/Colab Notebooks/content/sample_model (1).onnx")
#onnx_model = onnx.load("/content/drive/MyDrive/Colab Notebooks/content/your_model.onnx")

Extract weights from graph initializer

In [ ]:
INTIALIZERS  = onnx_model.graph.initializer
onnx_weights = {}
weights = []
for initializer in INTIALIZERS:
    W = numpy_helper.to_array(initializer)
    weights.insert(0,W)
    onnx_weights[initializer.name] = W
#print(weights[4].shape)

Mark the layer with Relu activation

In [ ]:
# Access the graph of the ONNX model
graph = onnx_model.graph

#activation relu - 1
act = []
i = 0
if graph.node[i].op_type == "Relu":
  #act.insert(0,1)
  act.append(1)
  i = i+1

while i < len(graph.node)-1 :
    node = graph.node[i]
    #print(node.op_type)
    if node.op_type == "MatMul":
      i = i+2
      if i >= len(graph.node)-1 :
        act.append(0)
        #act.insert(0,0)
        break
      node = graph.node[i]
      #print(node.op_type)
      if node.op_type == "Relu":
        act.append(1)
        #act.insert(0,1)
        i = i+1
      else:
        act.append(0)
        #act.insert(0,0)
      continue
    #i = i-1
    #print("=" * 40)
print(act)

[1, 1, 0]


In [ ]:
def get_hidden_units(weights):
    hidden_units = []
    for i in range(0,len(weights),2):
      hidden_units.append((weights[i].shape)[0])
    return hidden_units

In [ ]:
# Step 2: Create Z3 variables
input_size, _ = (weights[1].shape)
_, output_size = (weights[len(weights)-1].shape)
#num_hidden_layers = len(model.layers)
num_hidden_units = get_hidden_units(weights)
num_hidden_layers = len(act)
#print(num_hidden_layers)
print(num_hidden_units)

[32, 32, 18]


In [ ]:
input = []
'''
for i in range(input_size):
  input.append(1)
'''
input_file_path = '/content/drive/MyDrive/Colab Notebooks/content/config1_input.xlsx'

# Read the Excel sheet into a DataFrame
df_input_range = pd.read_excel(input_file_path, header=0)

for i in range(input_size):
  input.append(random.uniform(df_input_range['range min norm'][i], df_input_range['range max norm'][i]))
'''
input.append(random.uniform(-0.9513373022, 3.317523038))
input.append(random.uniform(-0.5, 0.5))
input.append(random.uniform(-1.330610499, 2.08708102))
for i in range(6):
  input.append(random.uniform(-0.5, 0.5))
'''

'\ninput.append(random.uniform(-0.9513373022, 3.317523038))\ninput.append(random.uniform(-0.5, 0.5))\ninput.append(random.uniform(-1.330610499, 2.08708102))\nfor i in range(6):\n  input.append(random.uniform(-0.5, 0.5))\n'

ONNX to Constraints

In [ ]:
#SMT file for volce
#file_path = "/content/drive/MyDrive/Colab Notebooks/content/simple_constraint.smt2"
file_path = "/content/drive/MyDrive/Colab Notebooks/content/constraint.smt2"
smt_output_file = open(file_path, "w")
#smt_output_file.write("(set-logic QF_LRA)\n(set-info :f1.smt2)\n(set-info :smt-lib-version 2.0)\n(set-info :status sat)\n")
smt_output_file.write("(set-logic QF_LRA)\n(set-info :smt-lib-version 2.0)\n")

51

In [ ]:
inputs = [Real('x{}'.format(i)) for i in range(input_size)]
outputs = [Real('y{}'.format(i)) for i in range(output_size)]
hidden_layers = []
activation_layers = []

for i in range(num_hidden_layers):
    hidden_layer = [Real('h{}{}'.format(i, j)) for j in range(num_hidden_units[i])]
    activation_layer = [Real('a{}{}'.format(i, j)) for j in range(num_hidden_units[i])]
    hidden_layers.append(hidden_layer)
    activation_layers.append(activation_layer)

for i in range(len(inputs)):
  smt_output_file.write("(declare-fun x"+str(i)+" () Real)\n")

'''
for i in range(len(outputs)):
  smt_output_file.write("(declare-fun y"+str(i)+" () Real)\n")

for i in range(num_hidden_layers):
  for j in range(num_hidden_units[i]):
    smt_output_file.write("(declare-fun h"+str(i)+str(j)+" () Real)\n")
    smt_output_file.write("(declare-fun a"+str(i)+str(j)+" () Real)\n")
'''

# Step 3: Build SMT Constraints
constraints = []
solver = Solver()

for i in range(num_hidden_units[0]):
  constraint = Sum([ inputs[j] * weights[1][j][i] for j in range(len(inputs))]) + weights[0][i]
  '''
  if weights[0][i] < 0:
    smt_code = f"(- "
  else:
    smt_code = f"(+ "
    # Summation of products
  '''
  smt_code = f"(+ "
  products = []
  for j in range(len(inputs)):
    if weights[1][j][i] >=0 :
      products.append(f"(* {inputs[j]} {weights[1][j][i]})")
    else:
      products.append(f"(* {inputs[j]} (- 0 {-1*weights[1][j][i]}))")

    # Add all the product terms to the SMT code
  smt_code += " ".join(products)
  if weights[0][i] >= 0:
    smt_code += f" {weights[0][i]})"
  else:
    smt_code += f" (- 0 {-1*weights[0][i]}))"
  '''
  # Add the bias term
  if weights[0][i] < 0:
    smt_code += f" {-1*weights[0][i]})"
  else:
    smt_code += f" {weights[0][i]})"
  '''
  '''
  smt_constraint = ""
  for j in range(len(inputs)):
    smt_constraint = str(inputs[j]) + "+" + smt_constraint
  #print(hidden_layers[0][i])
  '''
  solver.add(hidden_layers[0][i] == constraint)
  #smt_output_file.write("(define-fun {} () Real\n  ({}))\n".format(hidden_layers[0][i],constraint))
  smt_output_file.write("(define-fun {} () Real\n  {})\n".format(hidden_layers[0][i],smt_code))
  print(hidden_layers[0][i] == constraint)
  if act[0] == 1:
    solver.add(activation_layers[0][i] == If(hidden_layers[0][i] > 0, hidden_layers[0][i], 0))
    smt_output_file.write("(define-fun {} () Real\n  (ite (> {} 0) {} 0))\n".format(activation_layers[0][i],hidden_layers[0][i],hidden_layers[0][i]))
    print(activation_layers[0][i] == If(hidden_layers[0][i] > 0, hidden_layers[0][i], 0))
  else:
    solver.add(activation_layers[0][i] == hidden_layers[0][i])
    smt_output_file.write("(define-fun {} () Real\n  {})\n".format(activation_layers[0][i],hidden_layers[0][i]))
    print(activation_layers[0][i] == hidden_layers[0][i])

for k in range(1,num_hidden_layers):
  for i in range(num_hidden_units[k]):
    #print([weights[2*k][j][i] for j in range(num_hidden_units[k-1])])
    constraint = Sum([ activation_layers[k-1][j] * weights[2*k+1][j][i] for j in range(num_hidden_units[k-1])]) + weights[2*k][i]
    print(hidden_layers[k][i] == constraint)
    solver.add(hidden_layers[k][i] == constraint)
    #smt_output_file.write("(assert (= {} {}))\n".format(hidden_layers[k][i],constraint))
    #smt_output_file.write("(define-fun {} () Real\n  ({}))\n".format(hidden_layers[k][i],constraint))
    smt_code = f"(+ "
    # Summation of products
    products = []
    for j in range(len(inputs)):
      if weights[2*k+1][j][i] >= 0:
        products.append(f"(* {activation_layers[k-1][j]} {weights[2*k+1][j][i]})")
      else:
        products.append(f"(* {activation_layers[k-1][j]} (- 0 {-1*weights[2*k+1][j][i]}))")

    # Add all the product terms to the SMT code
    smt_code += " ".join(products)

    # Add the bias term
    if weights[2*k][i] >= 0:
      smt_code += f" {weights[2*k][i]})"
    else:
      smt_code += f" (- 0 {-1*weights[2*k][i]}))"

    smt_output_file.write("(define-fun {} () Real\n  {})\n".format(hidden_layers[k][i],smt_code))

    if act[k] == 1:
      solver.add(activation_layers[k][i] == If(hidden_layers[k][i] > 0, hidden_layers[k][i], 0))
      #smt_output_file.write("(assert (= {} ite (> {} 0) {} 0))\n".format(activation_layers[k][i],hidden_layers[k][i],hidden_layers[k][i]))
      smt_output_file.write("(define-fun {} () Real\n  (ite (> {} 0) {} 0))\n".format(activation_layers[k][i],hidden_layers[k][i],hidden_layers[k][i]))
      print(activation_layers[k][i] == If(hidden_layers[k][i] > 0, hidden_layers[k][i], 0))
    else:
      solver.add(activation_layers[k][i] == hidden_layers[k][i])
      #smt_output_file.write("(assert (= {} {}))\n".format(activation_layers[k][i],hidden_layers[k][i]))
      smt_output_file.write("(define-fun {} () Real\n  {})\n".format(activation_layers[k][i],hidden_layers[k][i]))
      print(activation_layers[k][i] == hidden_layers[k][i])

for i in range(num_hidden_units[-1]):
  solver.add(activation_layers[-1][i] == outputs[i])
  #smt_output_file.write("(assert (= {} {}))\n".format(activation_layers[-1][i],outputs[i]))
  smt_output_file.write("(define-fun {} () Real\n  {})\n".format(outputs[i],activation_layers[-1][i]))

'''
# Add constraints to the file
for c in solver.assertions():
  smt_output_file.write("(assert ({}))\n".format(c))
'''

'''
if solver.check() == sat:
    smt_model = solver.model()
    #input_values = [model.eval(input_var).as_decimal(10) for input_var in inputs]
    output_values = [smt_model.eval(output_var).as_decimal(10) for output_var in activation_layers[num_hidden_layers-1]]
    #print("Input:", input_values)
    print("Output:", output_values)
else:
    print("No solution found.")
'''

h00 ==
x0*4804729/1000000000 +
x1*15643071/5000000 +
x2*2087639/5000000 +
x3*4490639/50000000 +
x4*22366969/250000000 +
x5*2955329/25000000 +
x6*690023/40000000 +
x7*23217269/10000000000 +
x8*26096707/1000000000 +
-3366219/2500000
a00 == If(h00 > 0, h00, 0)
h01 ==
x0*17359473/50000000 +
x1*17064781/10000000 +
x2*-1038031/1562500 +
x3*-29735163/100000000 +
x4*-5132857/12500000 +
x5*-6604137/100000000 +
x6*-415547/1562500 +
x7*-23745263/2000000000 +
x8*3326901/5000000000 +
-3311057/5000000
a01 == If(h01 > 0, h01, 0)
h02 ==
x0*20344969/1000000000 +
x1*18359849/50000000 +
x2*4374551/25000000 +
x3*-7007931/10000000 +
x4*-20656551/100000000 +
x5*-10408189/50000000 +
x6*881347/250000 +
x7*1514151/20000000 +
x8*17281443/1000000000 +
-6179033/5000000
a02 == If(h02 > 0, h02, 0)
h03 ==
x0*-9577173/50000000 +
x1*10401/16000 +
x2*-4340729/25000000 +
x3*31904337/50000000 +
x4*69810313/100000000 +
x5*-2240013/12500000 +
x6*7180813/25000000 +
x7*32041/200000 +
x8*-79813/8000000 +
11070121/25000000
a03

'\nif solver.check() == sat:\n    smt_model = solver.model()\n    #input_values = [model.eval(input_var).as_decimal(10) for input_var in inputs]\n    output_values = [smt_model.eval(output_var).as_decimal(10) for output_var in activation_layers[num_hidden_layers-1]]\n    #print("Input:", input_values)\n    print("Output:", output_values)\nelse:\n    print("No solution found.")\n'

Including Input and Output constraints

In [ ]:
def smt_min_max(val):
  print(val)
  if not isinstance(val, str):
    if val < 0:
      return "(- 0 "+str(-1*val)+")"
    return str(val)
  if val[0:3] == "max":
    val = val[4:-1].split(',')
    const = "(ite (>= "+str(val[0])+" "+str(val[1])+") "+str(val[0])+" " + str(val[1])+")"
    for i in range(2,len(val)):
      const = "(ite (>= " + str(val[i]) + " " + const + ") " + str(val[i]) +")"
    return const
  elif val[0:3] == "min":
    val = val[4:-1].split(',')
    const = "(ite (>= "+str(val[0])+" "+str(val[1])+") "+str(val[0])+" " + str(val[1])+")"
    for i in range(2,len(val)):
      const = "(ite (>= " + str(val[i]) + " " + const + ") " + str(val[i]) +")"
    return const
  else:
    val = float(val)
    if val < 0:
      return "(- 0 "+str(-1*val)+")"
    return str(val)

In [ ]:
input = []
input_file_path = '/content/drive/MyDrive/Colab Notebooks/content/config1_input.xlsx'
output_file_path = '/content/drive/MyDrive/Colab Notebooks/content/config1_output.csv'

# Read the Excel sheet into a DataFrame
df_input_range = pd.read_excel(input_file_path, header=0)
df_output_range = pd.read_csv(output_file_path, delimiter=';',header=0)
#print(df_output_range.columns)
#print(df_output_range['Range min norm'])

for i in range(input_size):
  smt_output_file.write("(assert (> x" + str(i) + " "+smt_min_max(df_input_range['range min norm'][i])+"))\n")
  smt_output_file.write("(assert (< x" + str(i) + " "+smt_min_max(df_input_range['range max norm'][i])+"))\n")
  #input.append(random.uniform(df_input_range['range min norm'][i], df_input_range['range max norm'][i]))

for i in range(output_size):
  smt_output_file.write("(assert (> y" + str(i) + " "+smt_min_max(df_output_range['Range min norm'][i])+"))\n")
  smt_output_file.write("(assert (< y" + str(i) + " "+smt_min_max(df_output_range['Range max norm'][i])+"))\n")

smt_output_file.write("(check-sat)\n")

smt_output_file.write("(get-model)\n")
smt_output_file.close()

-0.9513373022
3.317523038
-0.5
0.5
1.330610499
2.08708102
-0.5
0.5
-0.5
0.5
-0.5
0.5
-0.5
0.5
-0.5
0.5
-0.5
0.5
0
10.0
max(y2,y3,y4,y5,y6,y7,y8)
10.0
0
10.0
0
10.0
0
10.0
0
10.0
0
10.0
0
10.0
0
10.0
-1.5
0.7
-0.3
0.0
-1.5
0.7
-1.5
0.7
-1.5
0.7
-1.5
0.7
-1.5
0.7
-1.5
0.7
-1.5
0.7


Rough Work

In [ ]:
session = onnxruntime.InferenceSession('/content/drive/MyDrive/Colab Notebooks/content/sample_model (1).onnx')
session.get_inputs()[0].shape
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

res = session.run([output_name], {input_name:[input]})
'''
print(len(list(res[0][0])))
print(res)
print(len(output_values))
'''
output_file_path = '/content/drive/MyDrive/Colab Notebooks/content/config1_output.xlsx'

# Read the Excel sheet into a DataFrame
df_output_range = pd.read_excel(output_file_path, header=0)

'''
range_check = []

for i in range(output_size):
  if (output_values >= df_output_range['Range min norm'][i] and output_values <= df_output_range['Range max norm'][i]):
    range_check.append(True)
  else:
    range_check.append(False)

# Create a DataFrame
df = pd.DataFrame({'Model Prediction': res[0][0], 'SMT solver': output_values, 'Range check': range_check})
'''

df = pd.DataFrame({'Model Prediction': res[0][0], 'SMT solver': output_values})
print(df)

In [ ]:
# Print the variable names and shapes

# Input variable names
print("Input variable names:")
for i, input_var in enumerate(inputs):
    print(f"Input {i+1}: {input_var}")
print()

# Shapes of weight arrays
print("Shapes of weight arrays:")
for i, weight_array in enumerate(weights):
    print(f"Weight array {i+1}: {weight_array.shape}")
print()

# Number of hidden units in each layer
print("Number of hidden units in each layer:")
print(num_hidden_units)
print()

# Shapes of hidden layer variables
print("Shapes of hidden layer variables:")
for i, hidden_layer in enumerate(hidden_layers):
    for j, hidden_var in enumerate(hidden_layer):
        print(f"Hidden layer {i+1}, unit {j+1}: {hidden_var}")
print()

# Shapes of activation layer variables
print("Shapes of activation layer variables:")
for i, activation_layer in enumerate(activation_layers):
    for j, activation_var in enumerate(activation_layer):
        print(f"Activation layer {i+1}, unit {j+1}: {activation_var}")


In [ ]:
# Create a new list to hold modified nodes
new_nodes = []

# Modify input names and create new nodes
for node in onnx_model.graph.node:
    new_input_names = [name.replace(':', '_') for name in node.input]
    new_node = onnx.helper.make_node(node.op_type, new_input_names, node.output, node.name)
    new_nodes.append(new_node)

# Replace old nodes with new nodes in the graph
del onnx_model.graph.node[:]
onnx_model.graph.node.extend(new_nodes)

for initialiser in onnx_model.graph.initializer:
    initialiser.name = initialiser.name.replace(':', '_')



In [ ]:
input_names = [input.name for input in onnx_model.graph.input]
print("Input names:", input_names)
keras_model = onnx_to_keras(onnx_model, input_names=keras_input_names)



In [ ]:
print(onnx_model.graph.node)

In [ ]:
print(onnx_model.graph.initializer)

In [ ]:
# Modify Nodes and Input Names
new_nodes = []

for node in onnx_model.graph.node:
    new_input_names = [name.replace(':', '_') for name in node.input]
    new_output_names = [name.replace(':', '_') for name in node.output]
    new_node = onnx.helper.make_node(node.op_type, new_input_names, new_output_names, name=node.name)
    new_nodes.append(new_node)

del onnx_model.graph.node[:]
onnx_model.graph.node.extend(new_nodes)

for initialiser in onnx_model.graph.initializer:
    initialiser.name = initialiser.name.replace(':', '_')

# Define Input Names for Keras Conversion
keras_input_names = [input.name for input in onnx_model.graph.input]

# Convert ONNX Model to Keras Model
model = onnx_to_keras(onnx_model, input_names=keras_input_names)